In [1]:
# GRABIM TEST
%load_ext autoreload
%autoreload 2
import numpy as np

import GRABIM as GRABIM


# Plot (external libraries)
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Legend, LegendItem
from bokeh.models import Arrow, NormalHead
from bokeh.models import ColumnDataSource, LabelSet, Text
output_notebook()

Loading BokehJS ...

In [2]:
# Problem input
f1 = 400e6; # [Hz] Start frequency
f2 = 1000e6; # [Hz] End frequency
m = 10; # Number of frequency samples
freq = np.linspace(f1, f2, m);

ZS = [20-10j, 20-20j, 20-10j, 25-5j, 30+10j, 25-10j, 30-10j, 20-10j, 20-30j, 30-20j]; # [Ohm] Source impedance
ZL = [120+30j, 120+30j, 115+20j, 100+30j, 120+50j, 100, 120+30j, 120+30j, 100+30j, 100+20j, 110+30j]; # [Ohm] Load impedance

In [3]:
# GRID SEARCH
code = ['CS', 'LP', 'CS', 'LP', 'CS', 'LP'];
[v_best, RL] = GRABIM.GridSearch(ZS, ZL, freq, m, code, 1, delta_X = 0.25)
print(v_best)
print("RLmax = ", "{:.2f}".format(RL), 'dB')

GRABIM.Plot_S21_S11_dB(ZS, ZL, code, v_best, freq)

[2.96613177e-11 1.17888879e-08 5.15436414e-12 1.71527307e-08
 3.54253921e-12 6.78403658e-08]
RLmax =  11.21 dB


In [4]:
def Objective_Function(x, *args):
    ZS = args[0]
    ZL = args[1]
    freq = args[2]
    code = args[3]
    # Calculate the reflection coefficient over the whole frequency span
    rho_k = GRABIM.LadderCircuitAnalysis.get_Input_Reflection_Coeff(ZS, ZL, code, x, freq);
    max_rho_k = 20*np.log10(np.max(np.abs(rho_k)));# Get the maximum
    return max_rho_k

In [5]:
from scipy.optimize import minimize

params = (ZS,  ZL, freq, code)
result = minimize(Objective_Function, v_best, args=params, method="nelder-mead",  options={'xatol':1e-3})
print("Status:", result['message'])
print("rho_max:", result['fun'], 'dB')
print("x_best = ", result['x'])
GRABIM.Plot_S21_S11_dB(ZS, ZL, code, result['x'], freq)

Status: Optimization terminated successfully.
rho_max: -11.373314698023647 dB
x_best =  [3.26105769e-11 1.15192992e-08 5.35890184e-12 1.72339592e-08
 3.90463013e-12 6.56237050e-08]
